In [1]:
%cd '/Users/abhilashdhal/Projects/'

/Users/abhilashdhal/Projects


## 1. Imports

In [2]:
import gevent.monkey
gevent.monkey.patch_all(thread=False, select=False)
import pandas as pd
import numpy as np

import os
from importlib import reload
from flatten_json import flatten
from tqdm import tqdm 
import src.Engines.gdc_engine as gdc_engine
import src.Connectors.gcp_bigquery_utils as gcp_bigquery_utils
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
reload(gcp_bigquery_utils)
reload(gdc_engine)

<module 'src.Engines.gdc_engine' from '/Users/abhilashdhal/Projects/src/Engines/gdc_engine.py'>

## 1. Initialize gdc_engine to get data from GDC

In [3]:

params = {
    'files.experimental_strategy': 'RNA-Seq', 
    'data_type': 'Gene Expression Quantification'
}

gdc_eng_inst = gdc_engine.GDCEngine(**params)


## 2. Fetch data from GDC for single or multiple primary sites and upload to bigquery

In [6]:

primary_sites = [
    'Esophagus'
]

## Initialize BigQueryUtils with your project
bq_utils = gcp_bigquery_utils.BigQueryUtils(project_id='rnaseqml')
table_id = 'rnaseqml.rnaseqexpression.expr_clustered'

## Give Schema of your table to be created or updated 
schema = [
    bigquery.SchemaField("case_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("file_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("expr_unstr_count", "INTEGER", mode="REPEATED"),
    bigquery.SchemaField("tissue_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("sample_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("primary_site", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("tissue_or_organ_of_origin", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("age_at_diagnosis", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("primary_diagnosis", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("race", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("gender", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("group_identifier", "INTEGER", mode="NULLABLE")
]

## Create table with partitioning and clustering
bq_utils.create_bigquery_table_with_schema(
    table_id=table_id, schema=schema, partition_field="group_identifier", clustering_fields=["primary_site", "tissue_type"]
)

## Specify the Kind of Downstream Analysis you want to perform
downstream_analysis = 'DE'
for site in tqdm(primary_sites):
    json_object = gdc_eng_inst.get_data_for_bq(site, downstream_analysis='DE', format='json')

    # Load data into BigQuery
    job = bq_utils.load_json_data(json_object, schema, table_id)
    job.result()  # Wait for the job to complete
    print(f"Data for {site} loaded successfully.")

I0000 00:00:1725299017.270191  632856 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1725299017.286906  632856 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Table Already Exists


  0%|          | 0/1 [00:00<?, ?it/s]

dict_keys(['files.experimental_strategy', 'data_type'])
file_id,file_name,submitter_id,experimental_strategy,data_type,platform,cases.case_id,cases.project.name,cases.project.project_id,cases.project.program.name,cases.project.program.program_id,cases.samples.sample_id,cases.diagnoses.last_known_disease_status,cases.diagnoses.primary_diagnosis,cases.diagnoses.tumor_stage,cases.diagnoses.tumor_grade,cases.diagnoses.treatments.treatment_or_therapy,cases.diagnoses.days_to_last_follow_up,cases.diagnoses.age_at_diagnosis,cases.diagnoses.days_to_death,cases.project.primary_site,analysis.workflow_type,cases.demographic.ethnicity,cases.demographic.gender,cases.demographic.race,cases.diagnoses.tissue_or_organ_of_origin,cases.exposures.bmi,cases.exposures.alcohol_history,cases.exposures.years_smoked,cases.samples.tissue_type,cases.samples.sample_type
dict_keys(['files.experimental_strategy', 'data_type'])
file_id,file_name,submitter_id,experimental_strategy,data_type,platform,cases.case_id,cases







100%|██████████| 50/50 [00:00<00:00, 80.45it/s]






100%|██████████| 50/50 [00:00<00:00, 94.53it/s]





100%|██████████| 1/1 [01:37<00:00, 97.06s/it]

Data for Esophagus loaded successfully.
